# Integrating Generative AI 


## File: gpt_response.py


In [ ]:
# Import the os module to access environment variables
import os

# Import the OpenAI module to interact with GPT
import openai

# Import dotenv to load environment variables from a .env file
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

# Get the OpenAI API key from environment variables
api_key = os.getenv("OPENAI_API_KEY")

# Create an OpenAI client using the API key
client = openai.OpenAI(api_key=api_key)

# Define a function to get a GPT response for a predicted disease
def get_gpt_response(predicted_disease, template_type="formal"):

    # Prepare the prompt to send to GPT asking for a medical summary
    prompt = f"""
    Give a short and clear medical summary for the disease: {predicted_disease}.
    Include:
    1. Description
    2. Treatment
    3. Prevention
    """

    # Send the prompt to the GPT-3.5-turbo model and get a response
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature=0.7  # Controls the creativity of the response
    )

    # Extract the content of the response
    reply = response.choices[0].message.content

    # Split the reply by new lines to separate each section
    lines = reply.split('\n')

    # Initialize empty variables to store each part of the response
    description = treatment = prevention = ""

    # Loop through the lines to extract description, treatment, and prevention
    for line in lines:
        if line.lower().startswith("1. description"):
            description = line.split(':', 1)[-1].strip()
        elif line.lower().startswith("2. treatment"):
            treatment = line.split(':', 1)[-1].strip()
        elif line.lower().startswith("3. prevention"):
            prevention = line.split(':', 1)[-1].strip()

    # Format the response using the formal template
    formal_response = f"""
    **Diagnosis:** {predicted_disease}

    **Description:**
    {description}

    **Treatment:**
    {treatment}

    **Prevention:**
    {prevention}
    """

    # Format the response using the friendly template
    friendly_response = f"""
    Looks like you're feeling some symptoms. Based on what you entered, you might have **{predicted_disease}**!

    Here's what it means:
    {description}

    To feel better:
    {treatment}

    To avoid it in the future:
    {prevention}
    """

    # Return the appropriate response based on the selected template type
    if template_type == "formal":
        return formal_response.strip()
    else:
        return friendly_response.strip()


## File: predict_disease.py


In [ ]:
# Import pandas for handling dataframes
import pandas as pd

# Import joblib to load the trained machine learning model
import joblib

# Import the GPT response function from the gpt_response module
from gpt_response import get_gpt_response

# Load the trained SVM model from the saved .pkl file
svm_model = joblib.load('svm_model.pkl')

# Load the training data to extract the list of all symptoms (feature names)
X_train = pd.read_csv("X_train.csv")
all_symptoms = X_train.columns.tolist()

# Define a function to predict the disease based on user-selected symptoms
def predict_disease(selected_symptoms):
    # Create a binary input list where 1 means the symptom is selected, 0 otherwise
    input_data = [1 if symptom in selected_symptoms else 0 for symptom in all_symptoms]
    
    # Convert the binary list into a DataFrame matching the training data format
    input_df = pd.DataFrame([input_data], columns=all_symptoms)

    # Use the trained model to predict the disease
    predicted_disease = svm_model.predict(input_df)[0]

    # Generate a GPT response for the predicted disease using the friendly template
    gpt_response = get_gpt_response(predicted_disease, template_type="friendly")
    
    # Return both the predicted disease and the GPT-generated response
    return predicted_disease, gpt_response


## File: symptom_checker_app.py


In [ ]:
# Import Streamlit to build the web interface
import streamlit as st

# Import pandas to handle CSV data
import pandas as pd

# Import joblib to load the trained machine learning model
import joblib

# Import the GPT response function
from gpt_response import get_gpt_response

# Import the function to predict the disease based on symptoms
from predict_disease import predict_disease  # Import the predict_disease function

# Load the trained SVM model from file
svm_model = joblib.load('svm_model.pkl')

# Load training data to get the list of all symptoms
X_train = pd.read_csv("X_train.csv")
all_symptoms = X_train.columns.tolist()

# Set the page configuration for the Streamlit web app
st.set_page_config(page_title="Symptom Checker with AI", layout="centered")

# Display the app title
st.title("🩺 Symptom Checker")

# Display a short description to guide the user
st.write("Select your symptoms and get a prediction with medical advice.")

# Create a multi-select dropdown for users to select symptoms
selected_symptoms = st.multiselect("Select your symptoms:", options=all_symptoms)

# When the user clicks the "Predict Disease" button
if st.button("Predict Disease"):
    # If no symptoms are selected, show a warning message
    if not selected_symptoms:
        st.warning("Please select at least one symptom.")
    else:
        # Call the prediction function and get the disease and GPT advice
        predicted_disease, gpt_response = predict_disease(selected_symptoms)
        
        # Show the predicted disease as a success message
        st.success(f"🧬 Predicted Disease: **{predicted_disease}**")

        # Show the GPT-generated medical advice
        st.markdown("### 💡 GPT Medical Advice")
        st.markdown(gpt_response)


## Comparison Between Formal and Friendly GPT Response Templates



## Objective of the Comparison:


The goal of this phase was to evaluate the effectiveness of two different GPT response templates: a formal medical explanation and a friendly conversational version. The comparison helps in identifying which format is more user-friendly and suitable for delivering AI-generated medical advice.



## Methodology:

After training the SVM model on a medical symptoms dataset and successfully predicting a disease (e.g., GERD), the predicted disease name was passed to the get_gpt_response() function using both template formats. The output from each was printed for direct side-by-side comparison.


## Templates Overview:

- Formal Template provides a structured, professional medical response. It is suited for documentation or clinical environments.

- Friendly Template offers the same medical content in a simpler, more approachable tone. It is designed to make non-expert users feel comfortable and engaged.



## Sample Output:

===== FORMAL TEMPLATE =====

**Diagnosis:** GERD

    **Description:**
    GERD, or Gastroesophageal Reflux Disease, is a chronic condition where stomach acid flows back into the esophagus, causing symptoms like heartburn, chest pain, and difficulty swallowing.

    **Treatment:**
    Treatment for GERD typically involves lifestyle changes such as avoiding trigger foods, eating smaller meals, and raising the head of the bed. Medications like antacids, H2 blockers, and proton pump inhibitors may also be prescribed to reduce acid production.

    **Prevention:**
    To prevent GERD, individuals can maintain a healthy weight, avoid trigger foods like spicy or fatty foods, and refrain from smoking. Eating smaller meals and avoiding eating close to bedtime can also help prevent symptoms of GERD.

===== FRIENDLY TEMPLATE =====

Looks like you're feeling some symptoms. Based on what you entered, you might have **GERD**!

    Here's what it means:
    GERD, or Gastroesophageal Reflux Disease, is a chronic condition where stomach acid flows back into the esophagus, causing symptoms like heartburn, chest pain, and difficulty swallowing.

    To feel better:
    Treatment for GERD typically involves lifestyle changes such as avoiding trigger foods, eating smaller meals, and raising the head of the bed. Medications like antacids, H2 blockers, and proton pump inhibitors may also be prescribed to reduce acid production.

    Stay safe!  
    To prevent GERD, individuals can maintain a healthy weight, avoid trigger foods like spicy or fatty foods, and refrain from smoking. Eating smaller meals and avoiding eating close to bedtime can also help prevent symptoms of GERD.

    ===== GPT-GENERATED COMPARISON TABLE =====



| Aspect            | Formal Template | Friendly Template |
|-------------------|-----------------|-------------------|
| Tone              | Formal          | Friendly          |
| Readability       | Moderate        | Easy              |
| Emotion           | Neutral         | Supportive        |
| Audience Suitability | Medical professionals or individuals seeking detailed information | General public or individuals seeking a simpler explanation 

## Observations:

- Both templates convey the same core information.

- The formal version is more professional but might be difficult for some users to understand.

- The friendly version improves readability and user comfort, especially for non-medical users.



We believe the friendly template is more engaging for regular users because of its empathetic tone, while the formal version is clearer for medical professionals. For our app, we found that the friendly version is more aligned with our target audience.


 ## Conclusion:

Based on clarity, tone, and ease of understanding, the friendly template is more effective for a general audience. However, in clinical settings or professional medical reports, the formal template would be more appropriate. The choice between them should depend on the target users.

